# recurrent_amplifications.ipynb
Count all oncogenes amplified in n >= 1 tumors. 

## TODO
- import data from data/Supplementary Tables.xlsx
- determine x-axis automatically instead of hard-coded in cancer_order variable
- determine y-axis automatically instead of hard-coded in chromosomal_gene_order variable

In [ ]:
library(readxl)
library(ggplot2)
library(tidyverse)
library(janitor)
library(writexl)
library(cowplot)
library(extrafont)
library(svglite)

extrafont::font_import(pattern="Arial",prompt=FALSE)
extrafont::loadfonts()

In [ ]:
# Data imports

deduplicated_amplicons <- readxl::read_excel("../data/combinedamplicons.xlsx") # Modified from the "_gene_list.tsv" output of AmpliconClassifier
ecDNA_amplicons <- deduplicated_amplicons %>% filter(grepl("ecDNA", amplicon_type)) #Subset the ecDNA amplicons
COSMIC <- read.csv("../data/COSMIC_allgeneset.csv") # CSV file from https://cancer.sanger.ac.uk/census

In [ ]:
#calculate total number of samples with amplification per gene per tumor type

count_gene_amps <- function(ac_tbl){
# Count gene amplifications from AmpliconClassifier output.
# Expects as input the columns "sample_ID" "sample_name" "amplicon_number" "amplicon_type" "gene" "gene_cn" "truncated" "cancer_type" "database"
# Returns: matrix of gene counts of size n_cancer_type x m_genes
  ac_tbl <- ac_tbl %>% 
    group_by(cancer_type, gene) %>% 
    summarize(count = n(), .groups="keep")
  colnames(ac_tbl) <- c("cancertype","gene","total_frequency")
  ac_tbl <- ac_tbl %>%
    pivot_wider(
      names_from = cancertype,
      values_from = total_frequency
    )
  return(ac_tbl)
}
# gene_freq_all_amplicons <- count_gene_amps(deduplicated_amplicons)
gene_freq_ecDNA <- count_gene_amps(ecDNA_amplicons)

In [ ]:
# Subset recurrently ecDNA-amplified oncogenes

subset_recurrently_ecDNA_amp_genes <- function(freq_tbl){
  # tbl should be genes (rows) x tumor types (cols). See count_gene_amps.
  rowsums <- tibble(freq_tbl$gene, rowSums(freq_tbl[,-1], na.rm = TRUE))
  colnames(rowsums) <- c("gene","count")
  rowsums <- rowsums %>% 
    filter(count > 1) %>%
    filter(gene %in% COSMIC$gene)
  freq_tbl <- freq_tbl %>% 
    filter(gene %in% rowsums$gene) %>%
    janitor::remove_empty(which = "cols")
  return(freq_tbl)
}

gene_x_tumor_amp_freq <- gene_freq_ecDNA %>% 
  subset_recurrently_ecDNA_amp_genes()
gene_x_tumor_amp_freq

In [ ]:
get_x_barplot_data <- function(sort="frequency"){
  # Count amplicons represented in each column. Note this is not a column sum of the heatmap, since an
  # amplification may have more than one amplified oncogene.
  tumor_amp_freq <- ecDNA_amplicons %>%
  filter(gene %in% gene_x_tumor_amp_freq$gene) %>%
  distinct(sample_name, .keep_all = TRUE) %>%
  group_by(cancer_type) %>%
  summarize(count = n()) #%>%
  # sort by ecDNA frequency, excluding other
  last_row <- tumor_amp_freq %>% slice_tail(n = 1)
  tumor_amp_freq <- tumor_amp_freq %>%
  slice_head(n = -1) %>%  # Exclude the last row
  arrange(-count) %>%       # Sort by the "Value" column
  bind_rows(last_row)      # Append the last row
  return(tumor_amp_freq)
}
get_y_barplot_data <- function(){
  # Count genes represented in each row
  gene_amp_freq <- tibble(gene_x_tumor_amp_freq$gene, rowSums(gene_x_tumor_amp_freq[,-1], na.rm = TRUE))
  colnames(gene_amp_freq) <- c("gene","count")
  gene_amp_freq <- gene_amp_freq %>% 
    #mutate(gene = factor(gene, levels=chromosomal_gene_order)) %>%
    arrange(count)
  return(gene_amp_freq)
}
## Sort by frequency
sort_heatmap_data <- function(hmd){
  # sort by column sums, keeping gene as first and other as last.
  ##column_sums <- c(Inf, colSums(hmd %>% ungroup() %>% select(-1, -ncol(hmd)), na.rm=TRUE),0) 
  ord <- get_x_barplot_data()$cancer_type
  # sort by rows
  row_sums <- rowSums(hmd %>% ungroup() %>% select(where(is.numeric)), na.rm=TRUE)
  return(hmd %>%
    select(all_of(ord)) %>% # Sort the tibble by order in x barplot
    arrange(row_sums) # Sort the tibble by decreasing row sums
    )
}
get_heatmap_data <- function(){
  gene_x_tumor_frequencytable   <- gene_x_tumor_amp_freq %>% 
    sort_heatmap_data %>%
    tidyr::pivot_longer(
      cols = NBL:other,
      names_to = c("cancer_type"),
      values_to = "count"
    ) #%>%
    #mutate(cancer_type = factor(cancer_type)) %>%
    #mutate(gene = factor(gene)) %>%
    #arrange(gene)
  return(gene_x_tumor_frequencytable)
}


cancer_order <- c("CPG","EP","ETMR","HGG","MB","NBL","OS","PB","RB","RHB","other")
chromosomal_gene_order <- c("PAX7","MYCN","FIP1L1","CHIC2","PDGFRA","KIT","KDR","TERT","TFEB","EGFR",
                            "CDK6","MET","RAD21","MYC","NDRG1","CCND1","BIRC3","CCND2","GLI1","DDIT3",
                            "CDK4","LRIG3","MDM2","PTPRB","FOXO1","RB1","GPC5","GAS7","NCOR1","FLCN",
                            "BRD4","CCNE1","TFPT","CNOT3") %>% rev

In [ ]:
#Integrated figure
# template @ https://rpubs.com/mgontar/215319

plot_gene_x_tumor_heatmap <- function(subpanel=FALSE){
  if(subpanel){
    plot_tag <- element_text(size=8,face = "bold", colour = "black")
    tag="b"
  }else{
    plot_tag <- NULL
    tag=NULL
  }
  mapping <- aes(x = fct_inorder(cancer_type), y = fct_inorder(gene), fill = count) 
  hm <- ggplot(data=get_heatmap_data(), mapping=mapping) +
    geom_raster() +
    #scale_fill_distiller(name = "Patient tumor count", palette = "Blues", direction = 1, na.value = "white", trans = 'log10', ) +
    scale_fill_gradient(name = "Patient tumor count", na.value="white", trans = 'log10',  low="skyblue",high="darkblue") +
    scale_x_discrete() + 
    labs(x="Tumor type", y="Oncogene", tag=tag) +
    theme_classic(base_size=7, base_family="Arial",) +
    theme(axis.text = element_text(size=7,colour="black"),
          axis.text.x = element_text(angle=45,vjust=0.5),
          plot.tag = plot_tag,
          legend.position = "bottom", 
          legend.direction = "horizontal",
          legend.key.size = unit(1,"cm"),
          ) 
  return(hm)
}

hm <- plot_gene_x_tumor_heatmap()
leg <- cowplot::get_legend(hm)
hm.clean <- hm + theme(legend.position="none")
hm

In [ ]:
# Create x axis barplot
x_barplot <- function(color=FALSE, labels=FALSE, subpanel=FALSE){
  # Color the bars or not
  if(color){
    mapping <- aes(x = fct_inorder(cancer_type), y = count, fill = count)
  }else{
    mapping <- aes(x = fct_inorder(cancer_type), y = count)
  }
  # label the y axis or not
  if(labels){
    axis_text_x <- element_text(angle=45,vjust=0.5)
    axis_ticks_x <- NULL
  }else{
    axis_text_x <- element_blank()
    axis_ticks_x <- element_blank()
  }
  # include subpanel labelling a, b, c etc.
  if(subpanel){
    plot_tag <- element_text(size=8,face = "bold", colour = "black")
    tag="a"
  }else{
    plot_tag <- NULL
    tag=NULL
  }
  
  bp.x <- ggplot(data = get_x_barplot_data(), mapping = mapping) + 
    geom_bar(stat = "identity", mapping) + 
    theme_classic(base_size=7, base_family="Arial") + 
    theme(
      axis.text = element_text(colour="black",size=7),
      plot.tag = plot_tag,
      axis.text.x = axis_text_x,
      axis.title.x = element_blank(),
      axis.ticks.x = axis_ticks_x,
      legend.position = "none") + 
    scale_fill_distiller(name = "Value", palette = "Blues", direction = 1, trans = 'log10' ) + 
    labs(x = "Cancer Type", y = "Patient tumors with frequently\necDNA-amplified oncogenes", tag=tag)
  return(bp.x)
}
bp.x <- x_barplot(color=FALSE,labels=TRUE)
bp.x

In [ ]:
# Create y axis barplot
y_barplot <- function(color=FALSE, labels=FALSE, subpanel=FALSE){
  # Color the bars or not
  if(color){
    mapping <- aes(x = count, y = fct_inorder(gene), fill = count)
  }else{
    mapping <- aes(x = count, y = fct_inorder(gene))
  }
  # label the y axis or not
  if(labels){
    axis_text_y <- NULL
    axis_ticks_y <- NULL
  }else{
    axis_text_y <- element_blank()
    axis_ticks_y <- element_blank()
  }
  # include subpanel labelling a, b, c etc.
  if(subpanel){
    plot_tag <- element_text(size=8,face = "bold", colour = "black")
    tag="c"
  }else{
    plot_tag <- NULL
    tag=NULL
  }
  
  bp.y <- ggplot(data = get_y_barplot_data(), mapping=mapping) + 
    geom_col(stat = "identity", mapping=mapping) + 
    scale_x_log10() +
    annotation_logticks(sides='b') +
    theme_classic(base_size=7, base_family="Arial") + 
    theme(
      axis.text = element_text(colour="black",size=7),
      plot.tag = plot_tag,
      axis.text.y = axis_text_y,
      axis.title.y = element_blank(),
      axis.ticks.y = axis_ticks_y,
      legend.position = "none") + 
    scale_fill_distiller(name = "Value", palette = "Blues", direction = 1, trans = 'log10' ) + 
    labs(x = "Patient tumors with frequently ecDNA-amplified oncogenes", tag=tag)
  return(bp.y)
}
bp.y <- y_barplot(color=FALSE,labels=TRUE,subpanel=TRUE)
bp.y

In [ ]:
assemble_plot <- function(){
  cowplot::plot_grid(
    bp.x, leg, hm.clean, bp.y,
    align = "hv",
    axis = "lrbt",
    nrow=2,
    ncol=2,
    rel_heights=c(1,2)
  )
}
bp.y <- y_barplot()
bp.x <- x_barplot()

assemble_plot()
ggsave(filename="amplicon_plot_grey_bar.png",path="out",dpi=300,width=7,height=7,units="in",bg="white")
ggsave(filename="amplicon_plot_grey_bar.svg",path="out",dpi=300,width=7,height=7,units="in")

In [ ]:
write_outputs <- function(filename){
  outdir <- file.path("out")
  dir.create(outdir, showWarnings = FALSE)
  path <- file.path(outdir,filename)
  write_xlsx(list(gene_x_tumor_amp_freq = get_heatmap_data(), 
                  tumor_amp_freq = get_x_barplot_data(),
                  gene_amp_freq = get_y_barplot_data()),
             path=path)
}
write_outputs('PedPanCancerAmpliconAnalysisOutputs.xlsx')